In [1]:
%cd ../airflow
%pwd

/home/welberthime/Development/fiap/cartolafc/airflow


'/home/welberthime/Development/fiap/cartolafc/airflow'

In [2]:
from include import RawExtractor

extractor = RawExtractor(
    base_url="https://api.github.com/repos/henriquepgomide/caRtola/contents/data",
    path="./data/raw",
)

extractor.extract_static_files("2020")
extractor.extract_dynamic_files("2020")

In [3]:
import pandas as pd
import uuid

In [18]:
partidas_df = pd.read_csv("./data/raw/2020/partidas/1.csv")

partidas_df.rename(
    columns={
        "home_team": "clubeMandanteID",
        "away_team": "clubeVisitanteID",
        "home_score": "mandantePlacar",
        "away_score": "visitantePlacar",
        "round": "rodada",
    },
    inplace=True,
)

def get_resultado(row):
    if row["mandantePlacar"] > row["visitantePlacar"]:
        return "Casa"
    elif row["mandantePlacar"] < row["visitantePlacar"]:
        return "Visitante"
    else:
        return "Empate"
    
partidas_df.drop_duplicates(inplace=True)
partidas_df["resultado"] = partidas_df.apply(get_resultado, axis=1)
partidas_df["partidaID"] = [uuid.uuid4().hex for _ in range(len(partidas_df))]
partidas_df["temporada"] = 2020

partidas_df = partidas_df[[
    "partidaID",
    "clubeMandanteID",
    "clubeVisitanteID",
    "mandantePlacar",
    "visitantePlacar",
    "rodada",
    "resultado",
    "temporada",
]]

partidas_df

,partidaID,clubeMandanteID,clubeVisitanteID,mandantePlacar,visitantePlacar,rodada,resultado,temporada
0,d0a0b8d23a854c948d82b9c45a7b392c,356,293,0,2,1,Visitante,2020
1,5488dab7362c44ba93dcdf503017b676,294,285,0,1,1,Visitante,2020
2,a19b3f57b18e416789fead6ba5cd8772,292,354,3,2,1,Casa,2020
3,b0d4a06f7684449a8d10e9e94d650e26,262,282,0,1,1,Visitante,2020
4,40a50f3155244990ac056b5049f82f3d,277,280,1,1,1,Empate,2020
...,...,...,...,...,...,...,...,...
380,543d90b2fe1a4885819545a6b36781e2,285,264,0,0,38,Empate,2020
382,de7e87039e0040d298c2066c3ad444ae,265,277,2,0,38,Casa,2020
384,ecac0e432bdc48348b7c12973fa65f8f,293,292,2,0,38,Casa,2020
386,47dbbc3324a74d5f906f40cd2b500972,354,263,2,1,38,Casa,2020


In [21]:
from glob import glob

file_list = glob("./data/raw/2020/rodada/*.csv")
scouts_df = pd.DataFrame()
for file in file_list:
    tmp_df = pd.read_csv(file)
    scouts_df = pd.concat([scouts_df, tmp_df], sort=True)

scouts_df.rename(
    columns={
        "atletas.clube_id": "clubeID",
        "atletas.atleta_id": "atletaID",
        "atletas.rodada_id": "rodada",
        "atletas.pontos_num": "pontos",
        "atletas.media_num": "pontosMedia",
        "atletas.preco_num": "preco", 
        "atletas.variacao_num": "precoVariacao",
    },
    inplace=True,
)

scouts_df["PE"] = None
scouts_df["RB"] = None

house_by_rounds = partidas_df[["partidaID", "rodada", "clubeMandanteID"]].rename(
    columns={"clubeMandanteID": "clubeID"}
)
visitor_by_rounds = partidas_df[["partidaID", "rodada", "clubeVisitanteID"]].rename(
    columns={"clubeVisitanteID": "clubeID"}
)
club_by_rounds = pd.concat([house_by_rounds, visitor_by_rounds])

scouts_df = scouts_df.merge(
    right=club_by_rounds,
    on=["rodada", "clubeID"],
)

scouts_df = scouts_df[[
    "clubeID",
    "partidaID",
    "atletaID",
    "pontos",
    "pontosMedia",
    "preco",
    "precoVariacao",
    "FS",
    "PE",
    "A",
    "FT",
    "FD",
    "FF",
    "G",
    "I",
    "PP",
    "RB",
    "FC",
    "GC",
    "CA",
    "CV",
    "SG",
    "DD",
    "DP",
    "GS",
]]

for column in scouts_df.columns:
    if column not in ["clubeID", "partidaID", "atletaID"]:
        scouts_df[column] = scouts_df[column].fillna(0).astype(float)

scouts_df

,clubeID,partidaID,atletaID,pontos,pontosMedia,preco,precoVariacao,FS,PE,A,...,PP,RB,FC,GC,CA,CV,SG,DD,DP,GS
0,373,ef729abb9d9144ee9b127ee4db7ba5b5,82758,-0.6,1.60,6.13,-0.22,3.0,0.0,0.0,...,0.0,0.0,10.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,373,ef729abb9d9144ee9b127ee4db7ba5b5,91203,-0.8,0.73,3.28,-0.29,8.0,0.0,0.0,...,0.0,0.0,7.0,0.0,1.0,0.0,1.0,0.0,0.0,0.0
2,373,ef729abb9d9144ee9b127ee4db7ba5b5,80370,3.0,5.46,8.98,-1.08,17.0,0.0,1.0,...,0.0,0.0,14.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
3,373,ef729abb9d9144ee9b127ee4db7ba5b5,87929,3.8,1.27,3.25,0.67,4.0,0.0,0.0,...,0.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,373,ef729abb9d9144ee9b127ee4db7ba5b5,89815,1.8,3.60,8.77,0.33,15.0,0.0,1.0,...,0.0,0.0,16.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
29109,267,7b1aa4a404484dd881324316108cbd90,80196,3.0,3.43,7.99,-1.19,28.0,0.0,0.0,...,0.0,0.0,16.0,0.0,4.0,0.0,6.0,0.0,0.0,0.0
29110,267,7b1aa4a404484dd881324316108cbd90,79631,3.8,5.28,13.44,-0.22,17.0,0.0,1.0,...,0.0,0.0,22.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
29111,267,7b1aa4a404484dd881324316108cbd90,99881,0.0,2.57,6.91,0.00,7.0,0.0,0.0,...,0.0,0.0,10.0,0.0,2.0,0.0,4.0,0.0,0.0,0.0
29112,267,7b1aa4a404484dd881324316108cbd90,105786,3.2,3.87,10.34,-1.43,55.0,0.0,2.0,...,0.0,0.0,20.0,0.0,3.0,1.0,0.0,0.0,0.0,0.0


In [7]:
posicoes_df = pd.read_csv("./data/raw/2020/posicoes_ids/1.csv")
posicoes_df.rename(
    columns={
        "Cod": "posicaoID",
        "Position": "nome",
        "abbr": "abreviacao",
    },
    inplace=True,
)

posicoes_df = posicoes_df[[
    "posicaoID",
    "nome",
    "abreviacao",
]]

posicoes_df

,posicaoID,nome,abreviacao
0,1,Goleiro,gol
1,2,Lateral,lat
2,3,Zagueiro,zag
3,4,Meia,mei
4,5,Atacante,ata
5,6,Técnico,tec


In [22]:
from glob import glob

file_list = glob("./data/raw/2020/rodada/*.csv")
atletas_df = pd.DataFrame()
for file in file_list:
    tmp_df = pd.read_csv(file)
    atletas_df = pd.concat([atletas_df, tmp_df], sort=True)

atletas_df.rename(
    columns={
        "atletas.clube_id": "clubeID",
        "atletas.atleta_id": "atletaID",
        "atletas.apelido": "apelido",
    },
    inplace=True,
)

atletas_df = atletas_df.merge(
    right=posicoes_df,
    left_on="atletas.posicao_id",
    right_on="abreviacao",
)

atletas_df = atletas_df[[
    "atletaID",
    "clubeID",
    "posicaoID",
    "apelido",
]]

atletas_df.drop_duplicates(inplace=True)
atletas_df.reset_index(drop=True, inplace=True)

atletas_df

,atletaID,clubeID,posicaoID,apelido
0,82758,373,5,Hyuri
1,92696,373,5,Renato Kayzer
2,100290,292,5,Lucas Venuto
3,96966,277,5,Arthur Gomes
4,103445,277,5,Kaio Jorge
...,...,...,...,...
1064,86820,373,2,Reginaldo
1065,80313,266,2,Gilberto
1066,110917,373,2,Carlos
1067,110980,356,2,Miguel


In [9]:
clubes_df = pd.read_csv("./data/raw/2020/times_ids/1.csv")
clubes_df.rename(
    columns={
        "id": "clubeID",
        "nome.cbf": "nome",
    },
    inplace=True,
)

clubes_df = clubes_df[["clubeID", "nome", "abreviacao"]]
clubes_df.drop_duplicates(subset=["clubeID"], keep="last", inplace=True)

clubes_df

,clubeID,nome,abreviacao
0,327,América - MG,AME
1,200,America - RN,OUT
2,373,Atlético - GO,ATL
3,282,Atlético - MG,ATL
6,293,Athlético-PR,ATL
7,314,Avaí - SC,AVA
8,265,Bahia - BA,BAH
9,263,Botafogo - RJ,BOT
10,203,Brasiliense,BRA
13,354,Ceará - CE,CEA
